# Small

In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', numThreads=4, quiet=True)
import simuPOP as sim
import pandas as pd
from saegus import breed, operators, simulate, analyze, parse, parameters
import shelve
import numpy as np
import random
import h5py
import collections as col
np.set_printoptions(suppress=True, precision=3)

In [2]:
small = analyze.Study('small')

In [3]:
run_id = 'small'
generations_of_random_mating = 10
number_of_qtl = 20
number_of_replicates = 10
founders = [[2, 26], [3, 25], [4, 24], [5, 23]]
os_per_pair = 500
recombination_rates = [0.01]*1478

In [4]:
prefounders = sim.loadPopulation('bia_prefounders.pop')

In [5]:
prefounders.infoFields()

('ind_id',
 'father_id',
 'mother_id',
 'fitness',
 'p',
 'g',
 'generation',
 'replicate')

In [6]:
sim.tagID(prefounders, reset=True)

In [7]:
prefounders.popSize()

26

In [8]:
multi_prefounders = sim.Simulator(prefounders, 10, stealPops=False)

In [9]:
magic = breed.MAGIC(multi_prefounders, founders, recombination_rates)

In [10]:
magic.generate_f_one(founders, os_per_pair)

In [11]:
mrc = breed.MultiRandomCross(multi_prefounders, 4, 500)

In [12]:
mother_choices, father_choices = mrc.determine_random_cross()

In [13]:
multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(
    mother_choices, father_choices)

In [14]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

In [15]:
final_mrc = breed.MultiRandomCross(multi_prefounders, 2, 1000)

In [16]:
final_mothers, final_fathers = final_mrc.determine_random_cross()

In [17]:
final_multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(
    final_mothers, final_fathers)

In [18]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(final_multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

# Random Mating Phase

In [19]:
multi_prefounders.evolve(
    matingScheme=sim.RandomMating(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
        subPopSize=[2000]),
    gen=10,
)

(10, 10, 10, 10, 10, 10, 10, 10, 10, 10)

In [20]:
sample_size = 200

In [21]:
sample_library = small.collect_samples(multi_prefounders, [200])

In [22]:
for rep_id, sample_list in sample_library.items():
    sim.stat(sample_list[0], numOfSegSites=sim.ALL_AVAIL, vars=['numOfSegSites', 'segSites'])
    sim.stat(sample_list[0], alleleFreq=sim.ALL_AVAIL)

In [23]:
sample = sample_library[0][0]

In [24]:
astates = small.gather_allele_data(sample)

In [25]:
alleles = np.array([astates[:, 1], astates[:, 2]]).T

In [26]:
segregating_loci = np.array(sample.dvars().segSites)

In [27]:
trait = parameters.Trait()

In [28]:
qtl = sorted(list(random.sample(list(segregating_loci), 20)))

In [29]:
allele_effects = trait.construct_allele_effects_table(alleles, qtl, random.expovariate, 1)

In [30]:
ae_array = trait.construct_ae_array(allele_effects, qtl)

# Storing Data

In [31]:
small_data = h5py.File('small_data.hdf5')

In [32]:
small_data['allele/states'] = astates
small_data['segregating_loci'] = segregating_loci
small_data['qtl'] = np.array(qtl)
small_data['allele/effects'] = allele_effects

In [33]:
for rep, sample_list in sample_library.items():
    small_data['allele/frequency/replicate/' + str(rep)] = small.gather_allele_frequencies(sample_list[0], astates)
    operators.calculate_g(sample_list[0], ae_array)
    operators.calculate_error_variance(sample_list[0], 0.7)
    operators.calculate_p(sample_list[0])
    small_data['trait/g/replicate/' + str(rep)] = np.array([sample_list[0].indInfo('ind_id'), 
                                                            sample_list[0].indInfo('g')]).T
    small_data['trait/p/replicate/' + str(rep)] = np.array([sample_list[0].indInfo('ind_id'),
                                                          sample_list[0].indInfo('p')]).T
    

In [34]:
small_data['trait'].attrs['heritability'] = np.array([0.7])

# Fixing TASSEL Input Function

In [35]:
minor_alleles = np.array(small_data['allele/states'])[:, 3]

In [36]:
name = 'small_0'

In [37]:
minor_allele_frequencies = np.array(small_data['allele/frequency/replicate/0'])[segregating_loci, 3]

In [38]:
gwas = analyze.GWAS(sample_library[0][0], 
                    segregating_loci,
                   minor_alleles,
                   'small')

In [39]:
cm = gwas.calculate_count_matrix()

In [40]:
ps, svd = gwas.pop_struct_eigendecomp(cm)

In [41]:
ps[1]*svd[:, 1]

array([-0.027, -0.484, -0.272, -0.188, -0.786,  0.702, -0.107, -0.643,
       -0.309, -0.45 ,  0.498,  0.342,  0.13 , -0.861,  0.437, -0.585,
        0.177,  0.129, -0.171, -0.32 ,  0.01 , -0.431, -0.024, -0.102,
       -0.008,  0.323,  0.561,  0.22 ,  0.381,  0.622, -0.104,  0.532,
        0.052,  0.292, -0.41 ,  0.359, -0.672,  0.394, -0.193,  0.917,
        0.49 , -0.262,  0.135, -0.096,  0.175, -0.097, -0.043,  0.447,
       -0.385, -0.055,  0.003, -0.185,  0.567, -0.036, -0.211,  0.13 ,
       -0.115, -0.01 , -0.023,  0.336,  0.116,  0.502,  0.301,  0.005,
       -0.366, -0.115,  0.114,  0.305, -0.203,  0.341, -0.118, -1.03 ,
        0.357, -1.007,  0.214,  0.02 , -0.024,  1.06 ,  0.036, -0.155,
       -0.381,  0.203, -0.316,  0.255,  0.439,  0.424,  0.272,  0.355,
       -0.155,  0.613, -0.288,  0.395,  0.112,  0.276,  0.142, -0.234,
        0.268, -0.327,  0.337, -0.043,  0.459,  0.359, -0.363,  0.405,
       -0.406,  0.151, -0.058, -0.075,  0.343,  0.004, -0.148, -0.06 ,
      

In [53]:
popstr = gwas.population_structure_formatter(ps, 
                                             svd, 
                                             number_of_pcs=2, 
                                             pop_struct_file_name='small_0_population_structure.txt')

In [46]:
km = gwas.calc_kinship_matrix(cm, minor_allele_frequencies, kinship_matrix_file_name='small_0_kinship_matrix.txt')

In [54]:
gwas.trait_formatter(trait_file_name='small_0_trait_vector.txt')

In [55]:
gwas.single_gen_multi_rep_tassel_config(0, 'gwas_pipeline.xml')

In [ ]:
mino

In [57]:
small.run_id

'small'

In [58]:
for rep, sample_list in sample_library.items():
    name = small.run_id + '_' + str(rep)
    minor_allele_fs = np.array(small_data['allele/frequency/replicate/' + str(rep)])[segregating_loci, 3]
    gwas = analyze.GWAS(sample_list[0], segregating_loci, minor_alleles, 'small')
    cm = gwas.calculate_count_matrix(count_matrix_file_name=name+'_count_matrix.txt')
    ps, svd = gwas.pop_struct_eigendecomp(cm)
    gwas.population_structure_formatter(ps, svd, number_of_pcs=2, pop_struct_file_name=name+'_structure_matrix')
    gwas.trait_formatter(trait_file_name=name+'_phenotype_vector.txt')
    gwas.calc_kinship_matrix(cm, minor_allele_fs, kinship_matrix_file_name=name+'_kinship_matrix.txt')
    gwas.hapmap_formatter(hapmap_file_name=name+'_simulated_hapmap.txt')
    gwas.single_gen_multi_rep_tassel_config(rep, 'gwas_pipeline.xml', output_prefix=name+'_out_')

# Correcting the Kinship Matrix Calculation

In [ ]:
M = np.matrix(cm - 1)
P = 2*(segmin_af - 0.5)

In [ ]:
Z = M - P

In [ ]:
scaling_terms = np.zeros((943))

In [ ]:
for idx, prob in enumerate(segmin_af):
    scaling_terms[idx] = 2*prob*(1 - prob)

In [ ]:
scaling_factor = sum(scaling_terms)

In [ ]:
G = (Z*Z.T)/scaling_factor

In [ ]:
G

In [ ]:
np.array(small_data['allele/frequencies/replicate/0'])

In [ ]:
minor_allele_frequencies = np.array(small_data['allele/frequency/replicate/0'])[segregating_loci, 3]

In [ ]:
segmin_af = minor_allele_frequencies[segregating_loci]

In [ ]:
np.argmax(segmin_af)

In [ ]:
V = ((-1)*count_matrix) + 1

In [ ]:
V = cm

In [ ]:
V

In [ ]:
P = np.array([self.pop.dvars().alleleFreq[locus][allele]
                  for locus, allele in zip(self.segregating_loci,
                       self.segregating_minor_alleles)])

In [ ]:
P = np.array(small_data['allele/frequency/replicate/0'])[segregating_loci, 3]

In [ ]:
P = 1 - P

In [ ]:
Z = np.zeros((self.pop.popSize(),
                        len(self.segregating_loci)))



In [ ]:
Z = np.zeros((200, 943))

In [ ]:
G = np.zeros((self.pop.popSize(), self.pop.popSize()), dtype=np.float)

In [ ]:
G = np.zeros((200, 200))

In [ ]:
for i in range(self.pop.popSize()):
    Z[i, :] = V[i, :] - 2*(P - 0.5)

In [ ]:
for i in range(200):
    Z[i, :] = V[i, :] - 2*(P - 0.5)

In [ ]:
Z

In [ ]:
Z

In [ ]:
for i in range(self.pop.popSize()):
    for j in range(self.pop.popSize()):
        G[i, j] = np.sum(Z[i, :]*Z.T[:, j])

In [ ]:
for i in range(200):
    G[i, i] = np.sum(Z[i, :]*Z.T[:, i])

In [ ]:
Z[:, 0]

In [ ]:
Z.T[0, :] == Z[:, 0]

In [ ]:
G

In [ ]:
G

In [ ]:
(Z*Z).shape

In [ ]:
eigenvectors = [ps[i]*svd[i] for i in range(2)]

In [ ]:
eigvals = np.array(ps, dtype=np.float_)

In [ ]:
np.sort_complex(ps)

In [ ]:
svd.dtype

In [ ]:
eigenvectors[0].dtype

In [ ]:
ps.dtype

In [ ]:
svd.dtype

In [ ]:
(ps[0]*svd[0]).dtype

In [ ]:
svd.dtype

In [ ]:
ps.dtype

In [ ]:
ps[0]*svd[:, 0]

In [ ]:
cm.dtype

In [ ]:
minalls[segregating_loci]

In [ ]:
small_data.

In [ ]:
indir = "/home/vakanas/tassel-5-standalone/input/"
outdir = "/home/vakanas/tassel-5-standalone/output/"
rep_id_name = "0"

In [ ]:
gwas = analyze.GWAS(meta_pop, list(range(meta_pop.totNumLoci())), run_id)

In [ ]:
ccm = gwas.calculate_count_matrix(minor_alleles, list(range(meta_pop.totNumLoci())))

In [ ]:
ps_svd = gwas.pop_struct_svd(ccm)

In [ ]:
name = run_id+'_'+rep_id_name

In [ ]:
gwas.population_structure_formatter(ps_svd, indir+name+'_structure_matrix.txt')

In [ ]:
int_to_snp_map = {0:'A', 1:'C', 2:'G', 3:'T', 4:'-', 5:'+'}

In [ ]:
locus_names = list(concordant_segregating_loci)

In [ ]:
alleles_column = ['NA']*len(concordant_segregating_loci)

In [ ]:
chromosomes = [meta_pop.chromLocusPair(locus)[0]+1 for locus in concordant_segregating_loci]

In [ ]:
gwas.hapmap_formatter(concordant_segregating_loci, alleles_column, 
                      locus_names, chromosomes, 
                      locus_names, 
                      indir+name+'_simulated_hapmap.txt')

In [ ]:
minor_allele_frequency_table = analyze.minor_allele_frequencies_table(
        meta_pop.dvars().alleleFreq, minor_alleles)

In [ ]:
minor_allele_frequencies = np.array(minor_allele_frequency_table.minor_frequency)

In [ ]:
minor_allele_frequencies

In [ ]:
gwas.calc_kinship_matrix(ccm, minor_allele_frequencies, indir+name+'_kinship_matrix.txt')

In [ ]:
gwas.trait_formatter(indir+name+'_trait_vector.txt')

In [ ]:
import xml.etree.ElementTree as ET
import lxml.etree as etree

In [ ]:
config_file_template = '/home/vakanas/BISB/rjwlab-scripts/saegus_project/devel/magic/1478/gwas_pipeline.xml'

In [ ]:
tree = ET.parse(config_file_template)
root = tree.getroot()
lxml_tree = etree.fromstring(ET.tostring(root))
lxml_root = lxml_tree.getroottree()

In [ ]:
lxml_root.find('fork1/h').text = indir+name+'_simulated_hapmap.txt'
lxml_root.find('fork2/t').text = indir+name+'_trait_vector.txt'
lxml_root.find('fork3/q').text = indir+name+'_structure_matrix.txt'
lxml_root.find('fork4/k').text = indir+name+'_kinship_matrix.txt'

lxml_root.find('combine6/export').text = outdir+name+'_out_'

In [ ]:
lxml_root.write("/home/vakanas/tassel-5-standalone/"+"R"+rep_id_name+'_'+
                run_id+'_'+"_sim_gwas_pipeline.xml",
                encoding="UTF-8",
                method="xml", 
                xml_declaration=True, 
                standalone='',
                pretty_print=True) 

# Run TASSEL at This Point

# Use R Qvalue package to get Qvalues

# Combine TASSEL, Qvalues and Other Information

In [ ]:
qvalues = pd.read_csv("/home/vakanas/tassel-5-standalone/output/epsilon_0_qvalues.txt", sep='\t')

In [ ]:
qvalues.index = list(concordant_segregating_loci)

In [ ]:
qvalues

In [ ]:
raw_gwas_results = pd.read_csv("/home/vakanas/tassel-5-standalone/output/epsilon_0_out_2.txt", sep='\t')

In [ ]:
raw_gwas_results.drop(0, axis=0, inplace=True)

In [ ]:
raw_gwas_results.drop('Trait', axis=1, inplace=True)

In [ ]:
raw_gwas_results.index = np.array(list(map(int, raw_gwas_results.Marker)))

In [ ]:
raw_gwas_results

In [ ]:
raw_gwas_results = raw_gwas_results.join(qvalues)

In [ ]:
minor_allele_frequencies

In [ ]:
mafrqs = analyze.minor_allele_frequencies_table(meta_pop.dvars().alleleFreq, minor_alleles)

In [ ]:
raw_gwas_results = raw_gwas_results.join(mafrqs.ix[concordant_segregating_loci, :])

In [ ]:
raw_gwas_results

In [ ]:
def tassel_results_tables(gwas_file_name, q_values_file_name, 
                              minor_allele_frequency_table, 
                              quantitative_allele_table):
    raw_gwas_results = pd.read_csv(gwas_file_name, sep='\t')
    raw_gwas_results.drop(0, axis=0, inplace=True)
    raw_gwas_results.drop('Trait', axis=1, inplace=True)
    raw_gwas_results.index = np.array(list(map(int, raw_gwas_results.Marker)))
    q_values = pd.read_csv(q_values_file_name, sep='\t')
    q_values.index = np.array(list(map(int, raw_gwas_results.Marker)))
    raw_gwas_results = raw_gwas_results.join(q_values)
    
    assert minor_allele_frequency_table.index.dtype == raw_gwas_results.index.dtype, "Indexes of these tables are different"
    
    raw_gwas_results = raw_gwas_results.join(minor_allele_frequency_table.ix[raw_gwas_results.index, :])
    
    assert quantitative_allele_table.index.dtype == raw_gwas_results.index.dtype, "Indexes of these tables are different"
    
    raw_gwas_results = raw_gwas_results.join(quantitative_allele_table.ix[raw_gwas_results.index, :])
    return raw_gwas_results

In [ ]:
pwd

In [ ]:
cd /home/vakanas/tassel-5-standalone/output/

In [ ]:
ls

In [ ]:
mafrqs = pd.read_csv('epsilon_0_maf_table.txt', sep='\t', index_col=0)

In [ ]:
mafrqs

In [ ]:
qtad = pd.read_csv('epsilon_0_quant_allele_table.txt', sep='\t', index_col=0)

In [ ]:
qtad

In [ ]:
super_table = tassel_results_tables('epsilon_0_out_2.txt', 'epsilon_0_qvalues.txt', mafrqs, qtad)

In [ ]:
super_table.ix[super_table.q < 0.05]

In [ ]:
super_table.ix[super_table.alpha_effect > 0]

In [ ]:
mg.multiple_sample_analyzer(meta_populations, qtl, allele_effects, 
                            minor_alleles, concordant_segregating_loci)

In [ ]:
analyze.store_allele_effect_frequency_tables(meta_population, alleles, 
                                             qtl,
                                             exponential_allele_effects,
                                            run_id, 'exponential')

In [ ]:
loci_conversions = shelve.open(run_id+'_loci_conversions')
saegus_to_tassel_loci = {}
tassel_to_saegus_loci = {}
for idx, locus in enumerate(concordant_segregating_loci):
    saegus_to_tassel_loci[locus] = idx
    tassel_to_saegus_loci[idx] = locus
loci_conversions['saegus_to_tassel'] = saegus_to_tassel_loci
loci_conversions['tassel_to_saegus'] = tassel_to_saegus_loci
loci_conversions.close()

In [ ]:
seg_loc_storage = shelve.open('segregating_loci_storage')
seg_loc_storage['bacchus'] = concordant_segregating_loci
seg_loc_storage.close()

In [ ]:
int_to_snp = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: '-', 5: '+'}
snp_to_int = {'A': 0, 'C': 1, '-': 4, 'G': 2, '+': 5, 'T': 3}
conv = shelve.open('synthesis_parameters')
conv['integer_to_snp'] = int_to_snp
conv['snp_to_integer'] = snp_to_int
conv.close()

In [ ]:
exponential_allele_effects_table = analyze.generate_allele_effects_table(qtl, alleles, 
                                                exponential_allele_effects, saegus_to_tassel_loci)

In [ ]:
analyze.remap_allele_frequency_table_loci(analyze.reload_allele_frequencies_table(run_id, 0, 250, 
                                                                                  'exponential'), 
                                          concordant_segregating_loci)

In [ ]:
analyze.write_multiple_sample_analyzer(sample_library, sample_sizes, qtl, alleles, 
                                       exponential_allele_effects, 0.7,  concordant_segregating_loci, 
                                       run_id=run_id, sub_run_id='_exponential', 
                                       allele_frequency_hdf=run_id+'_allele_frequency_storage.h5')

In [ ]:
multiple_sample_analyzer(meta_populations, qtl, allele_effects, minor_alleles, concordant_segregating_loci)

In [ ]:
import h5py

In [ ]:
with h5py.File('bia_allele_frequencies.hdf5') as biaf:
    reloaded_af = np.array(biaf[afname])

In [ ]:
fisegloc = list(concordant_segregating_loci)

In [ ]:
minor_allele_frequencies = reloaded_af[fisegloc]

In [ ]:
def write_super_tables(power_and_fpr_raw_data, sample_sizes, number_of_replicates, run_id, sub_run_id=''):
    for size in sample_sizes:
        for rep in range(number_of_replicates):
            name = run_id + '_' + sub_run_id + '_' + str(rep) + '_' + str(size) + '_super_table.txt'
            power_and_fpr_raw_data[size][rep].to_csv(name, sep='\t')

In [ ]:
expo_power_fpr_raw_data = analyze.collect_power_analysis_data(run_id, sample_sizes, number_of_replicates, concordant_segregating_loci, 'exponential')

In [ ]:
expo_power_fpr_raw_data[250]

In [ ]:
write_super_tables(expo_power_fpr_raw_data,
                  sample_sizes,
                  number_of_replicates,
                  'bacchus',
                  sub_run_id='exponential')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = study.calculate_power_fpr(expo_power_fpr_raw_data, sample_sizes, 
                                                                             number_of_replicates, number_of_qtl)

In [ ]:
expo_results

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('bacchus_exponential_mean_and_stdev_power_fpr.csv', sep='\t')

In [ ]:
geo_results

In [ ]:
geometric_allele_effects_table

In [ ]:
exponential_allele_effects_table

In [ ]:
expo_results.to_csv("bacchus_exponential_power_fpr_results.txt", sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([geo_results.mean(), geo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_geometric_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = full_icecrown.calculate_power_fpr(expo_power_fpr_raw_data,
                                                                                      sample_sizes,
                                                                                      number_of_replicates,
                                                                                      number_of_qtl)

In [ ]:
expo_results

In [ ]:
expo_results.to_csv('full_icecrown_exponential_power_fpr_results.txt', sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_exponential_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
write_super_tables(expo_power_fpr_raw_data, sample_sizes, number_of_replicates, run_id, 'exponential')

In [ ]:
geo_aggregate_estimated_actual = pd.DataFrame([np.array(geo_agg_estimated), np.array(geo_agg_actual)], index=['estimated', 'actual']).T

In [ ]:
geo_aggregate_estimated_actual['estimated'] = geo_aggregate_estimated_actual['estimated'].apply(np.fabs)

In [ ]:
geo_aggregate_estimated_actual

In [ ]:
geo_corr = geo_aggregate_estimated_actual['estimated'].corr(geo_aggregate_estimated_actual['actual'])

In [ ]:
geo_agg_estimated

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_corr

In [ ]:
pwd

In [ ]:
geo_aggregate_estimated_actual.to_csv('full_icecrown_geometric_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
agg_estimated = []
agg_actual = []

In [ ]:
for rep in reps:
    for size in sample_sizes:
        sutable = sutable_collection[rep][size]
        droppable = list(sutable.ix[sutable.ix[:, 'difference'] == 0.0].index)
        qtloci = sutable.drop(droppable, axis=0)
        agg_estimated.extend(list(qtloci['add_effect']))
        agg_actual.extend(list(qtloci['difference']))

In [ ]:
aggregate_estimated_actual = pd.DataFrame([np.array(agg_estimated), np.array(agg_actual)], index=['estimated', 'actual']).T

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
aggregate_estimated_actual

In [ ]:
correlation_actual_vs_effects = aggregate_estimated_actual['estimated'].corr(aggregate_estimated_actual['actual'])

In [ ]:
aggregate_estimated_actual.to_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
cd C:\tassel\output\full_icecrown\exponential

In [ ]:
expo_estimated_actual = pd.read_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t', index_col=0)

In [ ]:
expo_estimated_actual

In [ ]:
aggregate_estimated_actual

In [ ]:
, from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

In [ ]:
output_notebook()

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_x = aggregate_estimated_actual['estimated']
geo_y = aggregate_estimated_actual['actual']

In [ ]:
p = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(-0.2, 4))

In [ ]:
p.scatter(geo_x, y, x="Estimated", y="Actual")

p.xaxis.axis_label = "Estimated"
p.yaxis.axis_label = "Actual"

In [ ]:
show(p)

In [ ]:
expo

In [ ]:
p = figure(title="Estimated vs Actual Allele Effects - Geometric Series", title_text_font_size="16")

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4))

x = np.array(expo_estimated_actual['estimated'])
y = np.array(expo_estimated_actual['actual'])

expo_plot.xaxis.axis_label = "Estimated"
expo_plot.yaxis.axis_label = "Actual"

In [ ]:
expo_plot.scatter(x, y)

In [ ]:
show(expo_plot)

In [ ]:
from bokeh.io import hplot

In [ ]:
geo_plot = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(0, 4), y_range=(0, 4))

In [ ]:
geo_x = aggregate_estimated_actual['actual']
geo_y = aggregate_estimated_actual['estimated']

In [ ]:
geo_plot.xaxis.axis_label = "Actual"
geo_plot.yaxis.axis_label = "Estimated"
geo_plot.scatter(geo_x, geo_y, x="Actual", y="Estimated")

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4), y_range=(0, 4))

expo_x = np.array(expo_estimated_actual['actual'])
expo_y = np.array(expo_estimated_actual['estimated'])

expo_plot.xaxis.axis_label = "Actual"
expo_plot.yaxis.axis_label = "Estimated"
expo_plot.scatter(expo_x, expo_y)

In [ ]:
multi_plot = hplot(geo_plot, expo_plot)
show(multi_plot)

In [ ]:
output_file("multi_plot.png")

In [ ]:
ls